Kulikovska --- Test Task

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re
import xlrd

In [2]:
# path to driver chrome
chromedriver = 'C:\\Users\\Администратор\\AppData\\Local\\Temp\\7zO8E5ED3CD\\chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('headless')  
browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)

<ipython-input-2-ba38ce4e4320>:5: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)


In [3]:
  # make requests and find link using BeautifulSoup
r = requests.get("http://www.ukrstat.gov.ua/")
soup = BeautifulSoup(r.text) 
head=soup.find('frame',{'name': 'banner'})

  # recieve link on head
link_head='http://www.ukrstat.gov.ua/'+head.get('src')
print(link_head)

http://www.ukrstat.gov.ua/head.html


In [4]:
  # make requests and find link using Selenium
browser.get(link_head)

statistic_info_link = browser.find_element_by_xpath("//*[contains(text(), 'Статистична інформація')]").get_attribute("outerHTML")
a=re.search('clickMen(.+?)main', statistic_info_link).group(1)

    # recieve link on "Statistical information"
link_statistic='http://www.ukrstat.gov.ua/'+a[2:24]
print(link_statistic)

http://www.ukrstat.gov.ua/operativ/oper_new.html


In [5]:
browser.get(link_statistic)
 
    # recieve link on "Agriculture, forestry and fisheries"
link_agriculture = browser.find_element_by_partial_link_text("Сільське, лісове та рибне").get_attribute("href")
print(link_agriculture)

http://www.ukrstat.gov.ua/operativ/menu/menu_u/cg.htm


In [6]:
browser.get(link_agriculture)

    # recieve link on production volume
link_production_volumes = browser.find_element_by_partial_link_text("Обсяг виробництва, урожайність та зібрана").get_attribute("href")
print(link_production_volumes)

http://www.ukrstat.gov.ua/operativ/operativ2021/sg/ovuzpsg/Arh_ovuzpsg_2021_u.html


In [7]:
browser.get(link_production_volumes)

    # recieve link on all files and choose xls file for necessary date
link_september =browser.find_elements_by_tag_name("a")
for i in link_september:
    print(i.get_attribute("href"))
# The first 6 links will open pages with archive for 2021-2011 years.
# And the next 3 links will download necessary files about agriculture for 01.09.21, 01.08.21, 01.07.21

http://www.ukrstat.gov.ua/operativ/operativ2020/sg/ovuzpsg/Arh_ovuzpsg_2020_u.html
http://www.ukrstat.gov.ua/operativ/operativ2021/sg/ovuzpsg/Arh_ovuzpsg_2021_u.html
http://www.ukrstat.gov.ua/operativ/operativ2020/sg/ovuzpsg/Arh_ovuzpsg_2020_u.html
http://www.ukrstat.gov.ua/operativ/operativ2019/sg/ovuzpsg/Arh_ovuzpsg_2019_u.html
http://www.ukrstat.gov.ua/operativ/operativ2018/sg/ovuzpsg/Arh_ovuzpsg_u.html
http://www.ukrstat.gov.ua/druk/publicat/Arhiv_u/07/Arch_zv_bl.htm
http://www.ukrstat.gov.ua/operativ/operativ2021/sg/ovuzpsg/ovuzpsg_0921.xls
http://www.ukrstat.gov.ua/operativ/operativ2021/sg/ovuzpsg/ovuzpsg_0821.xls
http://www.ukrstat.gov.ua/operativ/operativ2021/sg/ovuzpsg/ovuzpsg_0721.xls
https://creativecommons.org/licenses/by/4.0/deed.uk


In [8]:
  # choose link to download file for 09.09.21
link_ovuzpsg_0921="http://www.ukrstat.gov.ua/operativ/operativ2021/sg/ovuzpsg/ovuzpsg_0921.xls"

  # read file, point out number of sheet that is needed and make cell without information to display as "Nan"
ovuzpsg_0921 = pd.read_excel(link_ovuzpsg_0921, na_values = ['-', '‒', 'к'], sheet_name = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,30,31,38,39,45,46])

In [9]:
  # write all our types of crop in order that they are in xls file
name_of_crop=['wheat','winter wheat','spring wheat','barley','winter barley','spring barley','winter rye','oat','buckwheat','millet','soya beans','sunflower seeds','fodder roots']

In [10]:
  # write loop to extract data from all sheets and merge all at the end in one DataFrame
i=-1
appended_data = []
for n in ovuzpsg_0921:
    if (n % 2 == 0 and n != 46) or n == 45:
        a=ovuzpsg_0921[n]
        
        #harvested area
          #find names of region
        obl=a.iloc[3:27,11]
        obl=obl.reset_index(drop=True)
        
          #find name of crop and convert it to df
        i=i+1
        crop_name=name_of_crop[i]
        crop=[]
        for x in range(24):
            crop.append(crop_name)
        crop=pd.DataFrame(crop)
        
          #find parameter and convert it to df
        name=[]
        col=a.iloc[1,2]
        for x in range(24):
            name.append(col)
        name=pd.DataFrame(name)
        
          #values for given parameter
        val=a.iloc[3:27,2]
        val=val.reset_index(drop=True)
        
          #merge region, name of crop, name of parameter and corresponding values
        b=pd.concat([obl,crop,name,val],axis=1)
        b.columns = ['shapeId', 'comdty', 'param', 'value']
        
        #volume of production
        name_2=[]
        col_2=a.iloc[1,3]
        val_2=a.iloc[3:27,3]
        val_2=val_2.reset_index(drop=True)
        
        for x in range(24):
            name_2.append(col_2)
        name_2=pd.DataFrame(name_2)
        b_2=pd.concat([obl,crop,name_2,val_2],axis=1)     
        b_2.columns = ['shapeId', 'comdty', 'param', 'value']            
        
        #yield
        name_3=[]
        col_3=a.iloc[1,4]
        val_3=a.iloc[3:27,4]
        val_3=val_3.reset_index(drop=True)
        
        for x in range(24):
            name_3.append(col_3)
        name_3=pd.DataFrame(name_3)
        b_3=pd.concat([obl,crop,name_3,val_3],axis=1) 
        b_3.columns = ['shapeId', 'comdty', 'param', 'value']
        
        #merge all three 
        one_sheet=pd.concat([b,b_2,b_3])
        
          # store all DataFrame in list        
        appended_data.append(one_sheet)
        
    else:
  
        a=ovuzpsg_0921[n]
    
          #find parameter(here is percent 2021 to 2020) and convert it to df
        percentage=[]
        percent=a.iloc[2,3]
        
        for x in range(24):
            percentage.append(percent)
        percentage=pd.DataFrame(percentage)
        
          #values for percent
        val_01=a.iloc[4:28,3]
        val_01=val_01.reset_index(drop=True)
          
            #find names of region
        obl_01=a.iloc[4:28,8]
        obl_01=obl_01.reset_index(drop=True)
        
          #merge region, name of crop, percent and corresponding values
        b_01=pd.concat([obl_01,crop,percentage,val_01],axis=1)
        b_01.columns = ['shapeId', 'comdty', 'param', 'value']
        
          # store all DataFrame in list
        appended_data.append(b_01)
  
  # see pd.concat documentation for more info
appended_data = pd.concat(appended_data)

In [11]:
  #information received from .xls file
data_for_21_xls=appended_data.reset_index(drop=True)
data_for_21_xls

,shapeId,comdty,param,value
0,Vinnytsya,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",322.4
1,Volyn,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",164.5
2,Dnipropetrovsk,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",556.6
3,Donetsk,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",382.7
4,Zhytomyr,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",140.4
...,...,...,...,...
1243,Kherson,fodder roots,2021 у % до / \n % to 2020,NaN
1244,Khmelnytskiy,fodder roots,2021 у % до / \n % to 2020,49
1245,Cherkasy,fodder roots,2021 у % до / \n % to 2020,68
1246,Chernivtsi,fodder roots,2021 у % до / \n % to 2020,NaN


In [18]:
from datetime import date
  #date now
now=[]
n=date.today()
for x in range(1248):
    now.append(n)
now=pd.DataFrame(now, columns=['dateRelease'])     
now['dateRelease'] = now['dateRelease'].astype('datetime64[ns]')

In [22]:
  #date the first of January 2021
date=[]
d='2021-01-01'
for x in range(1248):
    date.append(d)
date=pd.DataFrame(date, columns=['date'])
date['date'] = date['date'].astype('datetime64[ns]')

In [23]:
  # as a result receive table with necessary information
data_for_21=pd.concat([date,now,data_for_21_xls],axis=1)
data_for_21

,date,dateRelease,shapeId,comdty,param,value
0,2021-01-01,2021-10-10,Vinnytsya,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",322.4
1,2021-01-01,2021-10-10,Volyn,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",164.5
2,2021-01-01,2021-10-10,Dnipropetrovsk,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",556.6
3,2021-01-01,2021-10-10,Donetsk,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",382.7
4,2021-01-01,2021-10-10,Zhytomyr,wheat,"площа зібрана,\nтис.га / \nharvested area, \nt...",140.4
...,...,...,...,...,...,...
1243,2021-01-01,2021-10-10,Kherson,fodder roots,2021 у % до / \n % to 2020,NaN
1244,2021-01-01,2021-10-10,Khmelnytskiy,fodder roots,2021 у % до / \n % to 2020,49
1245,2021-01-01,2021-10-10,Cherkasy,fodder roots,2021 у % до / \n % to 2020,68
1246,2021-01-01,2021-10-10,Chernivtsi,fodder roots,2021 у % до / \n % to 2020,NaN
